In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
import mwparserfromhell as mw
import seaborn as sns
import matplotlib.pyplot as plt
import re
sns.set(style="whitegrid")
%matplotlib inline

%load_ext autoreload
%autoreload 2
import processing # this makes the "processing" folder available as a python module

from features.casualties import get_casualties, get_features
from utils import remove_ref, get_templates

In [ ]:
battles = json.load(open("../datasets/battle-fields-0.json"))
df = pd.DataFrame([b["infobox"] for b in battles if not b["infobox"].get("error")])
pattern = re.compile("^casualties[0-9]")
df = df[[b for b in df.columns if pattern.match(b)]]

In [ ]:
missed1 = get_casualties(df, df['casualties1'],'1')

In [ ]:
missed2 = get_casualties(df, df['casualties2'],'2')

In [ ]:
missed3 = get_casualties(df, df['casualties3'],'3')

In [ ]:
missed4 = get_casualties(df, df['casualties4'],'4')

In [ ]:
df.to_csv("casualties")

In [ ]:
print("For each casualties columns we have ", missed1, " ", missed2, " ", missed3, " ", missed4, " lines that contain a number but are not parsed.")


Which means that almost 100% of the values have been parsed in each column:

In [ ]:
toPlot = {'casualities1': 100-100*missed1/len(df), 'casualities2': 100-(100*missed2/len(df)), 'casualties3':100-(100*missed3/len(df)), 'casualties4':100-(100*missed4/len(df)) }

plt.bar(range(len(toPlot)), toPlot.values())
plt.title("Percent of parsed rows")
plt.xticks(range(len(toPlot)), toPlot.keys())
plt.show()

In [ ]:
pd.options.display.max_colwidth = 20
df[['casualties1', 'casualties_1', 'casualties2', 'casualties_2', 'casualties3', 'casualties_3', 'casualties4', 'casualties_4']].head(10)

But, we also observe that for a lot of battles, the casualties are not given numerically:

In [ ]:
print("Out of ", len(df), " battles")
df = df.replace(0, np.NaN)
print("Casualties1 has ", df['casualties_1'].isnull().sum(), "null values")
print("Casualties2 has ", df['casualties_2'].isnull().sum(), "null values")
print("Casualties3 has ", df['casualties_3'].isnull().sum(), "null values")
print("Casualties4 has ", df['casualties_4'].isnull().sum(), "null values")

toPlot = {'casualities1': 100-100*df['casualties_1'].isnull().sum()/len(df), 'casualities2': 100-(100*df['casualties_2'].isnull().sum()/len(df)), 'casualties3':100-(100*df['casualties_3'].isnull().sum()/len(df)), 'casualties4':100-(100*df['casualties_4'].isnull().sum()/len(df)) }

plt.bar(range(len(toPlot)), toPlot.values())
plt.title("Percent of battles with numeric values for casualties")
plt.xticks(range(len(toPlot)), toPlot.keys())
plt.show()

We also observe that almost 60% of the battles have numeric values for two combatants.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
#regarder devoir pour garder grandes valeurs
ax1.scatter(range(len(df)), df['casualties_1'], c='b', marker="s", label='casualties_1')
ax1.scatter(range(len(df)), df['casualties_2'], c='r', marker="o", label='casualties_2')
plt.title("Comparison between casualties of different combatants")
plt.legend(loc='upper left');
plt.show()

# moins de valeurs pour celui-la
# distribution et scatter
# comparaison casualties and strength

We observe th

In [ ]:
df["high_casualties_1"] = np.nan
df["high_casualties_2"] = np.nan
df.loc[df['casualties_1'] > 100000,'high_casualties_1'] = df['casualties_1']
df.loc[df['casualties_2'] > 100000,'high_casualties_2'] = df['casualties_2']

fig = plt.figure()
ax1 = fig.add_subplot(111)
#regarder devoir pour garder grandes valeurs
ax1.scatter(range(len(df)), df['high_casualties_1'], c='b', marker="s", label='casualties_1')
ax1.scatter(range(len(df)), df['high_casualties_2'], c='r', marker="o", label='casualties_2')
plt.title("Comparison between casualties of different combatants")
plt.legend(loc='upper left');
plt.show()



In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
#regarder devoir pour garder grandes valeurs
#ax1.scatter(range(len(df)), df['high_casualties_1'], c='b', marker="s", label='casualties_1')
ax1.scatter(df['casualties_1'], df['casualties_2'], c='r', marker="o", label='casualties_2')
plt.title("Comparison between casualties of different combatants")
plt.legend(loc='upper left');
plt.show()

In [ ]:
def densplot(columns, xlabel, title, axo):
    for i,v in enumerate(columns):
        sns.distplot(v, ax=axo, kde_kws={"label": i})
    axo.set_title(title)
    axo.set_xlabel(xlabel, fontsize=12)
    
def scatplot(xelem, yelem, xlabel, ylabel, title, polyfit=None):
    plt.scatter(xelem, yelem)
    if polyfit:
        plt.plot(np.unique(xelem), np.poly1d(np.polyfit(xelem, yelem, polyfit))(np.unique(xelem)), 'C2')
    plt.title(title)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.show()

In [ ]:
df.query('casualties_1 >100 and casualties_2>100')['casualties_1']

In [ ]:
cbt_1 = np.array(df["combatant_1"].dropna())
cbt_2 = np.array(df["combatant_2"].dropna())
cbt_3 = np.array(df["combatant_3"].dropna())
print(len(cbt_1), len(cbt_2), len(cbt_3))

cbt_all = np.concatenate((cbt_1, cbt_2, cbt_3))

all_cbt_names = [c for cl in cbt_all for c in cl]
names = pd.Series(all_cbt_names)
print(len(set(all_cbt_names)))

f, ax = plt.subplots(figsize=(6, 15))
counts = names.value_counts().sort_values(ascending=False)
counts = counts.head(50)
sns.barplot(x=counts, y=counts.index, ax=ax)